In [238]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [239]:
df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [240]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [241]:
df.shape

(768, 9)

In [242]:
X =df.drop('Outcome',axis=1)
y = df['Outcome']

In [243]:
X.shape

(768, 8)

In [244]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [245]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [246]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [214]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X_train,y_train,epochs=100,batch_size=32,validation_data = (X_test,y_test))

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.4752 - loss: 0.7934 - val_accuracy: 0.5844 - val_loss: 0.6981
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5941 - loss: 0.7050 - val_accuracy: 0.6429 - val_loss: 0.6487
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6273 - loss: 0.6729 - val_accuracy: 0.6753 - val_loss: 0.6090
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6604 - loss: 0.6282 - val_accuracy: 0.7208 - val_loss: 0.5798
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6981 - loss: 0.5926 - val_accuracy: 0.7532 - val_loss: 0.5556
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7212 - loss: 0.5676 - val_accuracy: 0.7662 - val_loss: 0.5358
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7173 - loss: 0.5439 - val_accuracy: 0.7987 - val_loss: 0.5200
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7280 - loss: 0.5280 - val_accuracy: 0.7987 - val_loss: 0.

In [215]:
!pip install keras-tuner
import kerastuner as kt

In [216]:
def build_model(hp):
  model = keras.Sequential()
  model.add(keras.layers.Dense(32,activation='relu',input_dim=8))
  model.add(keras.layers.Dense(1,activation='sigmoid'))
  optimizer = hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [217]:
tuner1 = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,project_name='Hamster')
tuner1.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Reloading Tuner from ./Hamster/tuner0.json


In [218]:
tuner1.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [219]:
tuner1.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [220]:
model = tuner1.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [221]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [222]:
model = tuner1.get_best_models(num_models=1)[0]

In [223]:
model.fit(X_train,y_train,batch_size=32, epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.7494 - loss: 0.5251 - val_accuracy: 0.7987 - val_loss: 0.5254
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7800 - loss: 0.5067 - val_accuracy: 0.8117 - val_loss: 0.5130
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7622 - loss: 0.5179 - val_accuracy: 0.8052 - val_loss: 0.5037
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7641 - loss: 0.5043 - val_accuracy: 0.7857 - val_loss: 0.4971
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7762 - loss: 0.4818 - val_accuracy: 0.7857 - val_loss: 0.4920
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7822 - loss: 0.4638 - val_accuracy: 0.7922 - val_loss: 0.4886
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7744 - loss: 0.4816 - val_accuracy: 0.7922 - val_loss: 0.4859
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7807 - loss: 0.4541 - val_accuracy: 

In [224]:
def build_model1(hp):
  model = keras.Sequential()
  units = hp.Int('units',min_value=8,max_value=128,step=8)
  model.add(keras.layers.Dense(units=units,activation='relu',input_dim=8))
  model.add(keras.layers.Dense(1,activation='sigmoid'))
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [225]:
tuner2 = kt.RandomSearch(build_model1,objective='val_accuracy',max_trials=5,project_name='Rat')
tuner2.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Reloading Tuner from ./Rat/tuner0.json


In [226]:
tuner2.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [227]:
tuner2.get_best_hyperparameters()[0].values

{'units': 88}

In [228]:
model = tuner2.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [229]:
model.fit(X_train,y_train,batch_size=32, epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7707 - loss: 0.4946 - val_accuracy: 0.7922 - val_loss: 0.4774
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7778 - loss: 0.4898 - val_accuracy: 0.7727 - val_loss: 0.4717
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7880 - loss: 0.4587 - val_accuracy: 0.7727 - val_loss: 0.4693
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7593 - loss: 0.4735 - val_accuracy: 0.7922 - val_loss: 0.4657
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7853 - loss: 0.4540 - val_accuracy: 0.7987 - val_loss: 0.4641
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7702 - loss: 0.4585 - val_accuracy: 0.7922 - val_loss: 0.4646
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7947 - loss: 0.4361 - val_accuracy: 0.7987 - val_loss: 0.4642
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7878 - loss: 0.4271 - val_accuracy: 0.79

In [230]:
def build_model(hp):
  model = keras.Sequential()
  model.add(keras.layers.Dense(72,activation='relu',input_dim=8))
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    model.add(keras.layers.Dense(72,activation='relu',input_dim=8))
  model.add(keras.layers.Dense(1,activation='sigmoid'))
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [231]:
tuner3 = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,project_name='Mouse')
tuner3.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Reloading Tuner from ./Mouse/tuner0.json


In [232]:
tuner3.get_best_hyperparameters()[0].values

{'num_layers': 6}

In [233]:
model = tuner3.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [234]:
model.fit(X_train,y_train,batch_size=32, epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.7780 - loss: 0.4964 - val_accuracy: 0.7597 - val_loss: 0.5589
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7730 - loss: 0.4872 - val_accuracy: 0.7792 - val_loss: 0.4974
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8171 - loss: 0.3827 - val_accuracy: 0.7857 - val_loss: 0.4723
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7925 - loss: 0.4485 - val_accuracy: 0.7468 - val_loss: 0.5253
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7992 - loss: 0.4465 - val_accuracy: 0.7727 - val_loss: 0.4948
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7935 - loss: 0.4049 - val_accuracy: 0.7857 - val_loss: 0.4761
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7948 - loss: 0.4299 - val_accuracy: 0.8052 - val_loss: 0.4708
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8030 - loss: 0.4254 - val_accuracy: 0.77

In [235]:
def build_model(hp):
  model = keras.Sequential()
  counter = 0
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    if counter == 0:
      model.add(
          Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                      activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']),input_dim=8),
          model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
      )
    else:
        model.add(
          Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                      activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid'])),
          model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
      )

    counter+=1
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta']),loss='binary_crossentropy',metrics=['accuracy'])
  return model


In [236]:
tuner4 = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,project_name='HedgeHog')
tuner4.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Reloading Tuner from ./HedgeHog/tuner0.json


In [237]:
tuner4.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units0': 72,
 'activation0': 'tanh',
 'dropout0': 0.9,
 'optimizer': 'adam',
 'units1': 120,
 'activation1': 'relu',
 'dropout1': 0.4,
 'units2': 88,
 'activation2': 'tanh',
 'dropout2': 0.6,
 'units3': 96,
 'activation3': 'relu',
 'dropout3': 0.7,
 'units4': 96,
 'activation4': 'tanh',
 'dropout4': 0.4,
 'units5': 56,
 'activation5': 'tanh',
 'dropout5': 0.4,
 'units6': 64,
 'activation6': 'sigmoid',
 'dropout6': 0.8}

In [247]:
model = tuner4.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [248]:
model.fit(X_train,y_train,batch_size=32, epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6300 - loss: 0.6475 - val_accuracy: 0.7338 - val_loss: 0.5534
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6406 - loss: 0.6248 - val_accuracy: 0.7143 - val_loss: 0.5473
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6497 - loss: 0.6286 - val_accuracy: 0.7338 - val_loss: 0.5435
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6787 - loss: 0.6172 - val_accuracy: 0.7078 - val_loss: 0.5526
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6487 - loss: 0.6364 - val_accuracy: 0.6688 - val_loss: 0.5692
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6459 - loss: 0.6377 - val_accuracy: 0.6753 - val_loss: 0.5689
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6580 - loss: 0.6344 - val_accuracy: 0.7078 - val_loss: 0.5602
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6535 - loss: 0.6486 - val_accuracy: 0.71